In [58]:
!pip install cartiflette
!pip install topojson

import numpy as np
import numpy.linalg as al
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
from cartiflette import carti_download

In [22]:
#Chargement dataframe initial

url = "https://data.sports.gouv.fr/api/explore/v2.1/catalog/datasets/equipements-sportifs/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
equipement = pd.read_csv(url, sep=";", low_memory=False)

Index(['Numéro de l'équipement sportif', 'Numéro de l'installation sportive',
       'Date de l'enquête', 'Nom de l'installation sportive',
       'SIRET Installation', 'Adresse', 'Code Postal', 'Commune nom',
       'Commune INSEE', 'Type de particularité de l'installation',
       ...
       'Accessibilité aux personnes en situation de handicap sensoriel - signalétique',
       'Accessibilité aux personnes en situation de handicap sensoriel aux tribunes',
       'Accessibilité aux personnes en situation de handicap sensoriel aux vestiaires',
       'Longitude', 'Latitude',
       'Type de particularité de l'installation (brute)', 'Activités', 'QPV',
       'QPV à 200 mètres', 'Catégorie'],
      dtype='object', length=116)

In [ ]:
#On conserve seulement les variables qui peuvent nous intéresser 

equipement.columns[:60]
cols = ["Nom de l'installation sportive", "Code Postal", "Commune nom", "Commune INSEE", "Département Code", "Département Nom", "Densite Catégorie", "Nom de l'équipement sportif", "Type d'équipement sportif", "Longitude", "Latitude"]
equipement = equipement[cols]
equipement.head(6)

In [61]:
#On tente de faire une carte de france des équipements sportifs 

equipement = equipement.dropna(subset=["Longitude", "Latitude"])

gdf_pts = gpd.GeoDataFrame(
    equipement,
    geometry=gpd.points_from_xy(equipement["Longitude"], equipement["Latitude"]),
    crs="EPSG:4326"  # WGS84 (lat/lon)
)

shp_communes = carti_download(
      values = ["France"],
      crs = 4326,
      borders = "DEPARTEMENT",
      vectorfile_format="topojson",
      simplification=50,
      filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
      source="EXPRESS-COG-CARTO-TERRITOIRE",
      year=2022)



/opt/python/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: Invalid range specified 
  return ogr_read(


DataSourceError: id INSEE_DEP    PAYS LIBELLE_DEPARTEMENT  POPULATION  \0    None        43  France         Haute-Loire      227570   1    None        65  France     Hautes-Pyrénées      229567   2    None        29  France           Finistère      915090   3    None        22  France       Côtes-d'Armor      600582   4    None        38  France               Isère     1271166   ..    ...       ...     ...                 ...         ...   100  None       971  France          Guadeloupe      384239   101  None       972  France          Martinique      364508   102  None       973  France              Guyane      281678   103  None       974  France          La Réunion      861210   104  None       976  France             Mayotte      256518                                  SOURCE  \0    IGN:EXPRESS-COG-CARTO-TERRITOIRE   1    IGN:EXPRESS-COG-CARTO-TERRITOIRE   2    IGN:EXPRESS-COG-CARTO-TERRITOIRE   3    IGN:EXPRESS-COG-CARTO-TERRITOIRE   4    IGN:EXPRESS-COG-CARTO-TERRITOIRE   ..    

In [65]:
url2 = "https://www.data.gouv.fr/api/1/datasets/r/6a782ef9-8ad6-4e66-832d-338b1041a42d"
elus = pd.read_csv(url2, sep = ";", low_memory = False)



"""# 1) Construire les paires (Voix i, Nuance liste i) réellement présentes
pairs = [
    (f"Voix {i}", f"Nuance liste {i}")
    for i in range(1, 39)
    if f"Voix {i}" in df.columns and f"Nuance liste {i}" in df.columns
]
if not pairs:
    raise ValueError("Aucune paire 'Voix i' / 'Nuance liste i' trouvée dans les colonnes.")

voix_cols   = [v for v, n in pairs]
nuance_cols = [n for v, n in pairs]

# 2) Convertir les colonnes Voix en numérique
V = df[voix_cols].apply(pd.to_numeric, errors="coerce")

# 3) Trouver, par ligne, la position de la Voix max (en remplaçant NaN par -1 pour ne pas gagner)
V_filled = V.fillna(-1)
pos_max = V_filled.to_numpy().argmax(axis=1)         # position de la colonne gagnante parmi voix_cols
voix_max = V.max(axis=1)                             # valeur max réelle (NaN si toutes NaN)

# 4) Récupérer la nuance à la même position
nuance_array = df[nuance_cols].to_numpy()
nuance_gagnante = nuance_array[np.arange(len(df)), pos_max]

# (Optionnel) Colonnes d'identification de la commune si présentes
id_cols = [c for c in ["Code commune", "Libellé commune", "Nom commune", "Commune"] if c in df.columns]

# 5) Résultat final
resultat = pd.DataFrame({
    **({c: df[c] for c in id_cols}),   # ajoute les colonnes d'identité si elles existent
    "Voix_max": voix_max,
    "Nuance_gagnante": nuance_gagnante
})


# Exemple d’aperçu
print(resultat.head())
"""

'# 1) Construire les paires (Voix i, Nuance liste i) réellement présentes\npairs = [\n    (f"Voix {i}", f"Nuance liste {i}")\n    for i in range(1, 39)\n    if f"Voix {i}" in df.columns and f"Nuance liste {i}" in df.columns\n]\nif not pairs:\n    raise ValueError("Aucune paire \'Voix i\' / \'Nuance liste i\' trouvée dans les colonnes.")\n\nvoix_cols   = [v for v, n in pairs]\nnuance_cols = [n for v, n in pairs]\n\n# 2) Convertir les colonnes Voix en numérique\nV = df[voix_cols].apply(pd.to_numeric, errors="coerce")\n\n# 3) Trouver, par ligne, la position de la Voix max (en remplaçant NaN par -1 pour ne pas gagner)\nV_filled = V.fillna(-1)\npos_max = V_filled.to_numpy().argmax(axis=1)         # position de la colonne gagnante parmi voix_cols\nvoix_max = V.max(axis=1)                             # valeur max réelle (NaN si toutes NaN)\n\n# 4) Récupérer la nuance à la même position\nnuance_array = df[nuance_cols].to_numpy()\nnuance_gagnante = nuance_array[np.arange(len(df)), pos_max]\n\n#